In [ ]:
!pip install --upgrade  optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: transformers
    Foun

In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 100.0 MB/s eta 0:00:00


In [ ]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 102.5 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 262.2 kB/s eta 0:00:00


In [ ]:
!pip install datasets

In [ ]:
!pip install --upgrade  transformers

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.4
    Uninstalling transformers-4.43.4:
      Successfully uninstalled transformers-4.43.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1.21.4 requires transformers[sentencepiece]<4.44.0,>=4.29.0, but you have transformers 4.44.2 which is incompatible.


In [ ]:
from huggingface_hub import login
login()

In [ ]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import QuantizationConfig,QuantFormat,QuantizationMode
from optimum.onnxruntime import ORTModelForSequenceClassification,ORTModelForFeatureExtraction
from pathlib import Path
from transformers import AutoTokenizer, pipeline, DistilBertTokenizer, DistilBertForSequenceClassification, EvalPrediction
from tqdm import tqdm
import time
from evaluate import evaluator
from datasets import load_dataset, load_metric
import numpy as np


In [ ]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import QuantizationConfig,QuantFormat,QuantizationMode
from optimum.onnxruntime import ORTModelForSequenceClassification
from pathlib import Path
from transformers import AutoTokenizer, pipeline, DistilBertTokenizer, DistilBertForSequenceClassification, EvalPrediction
from tqdm import tqdm
import time
from evaluate import evaluator
from datasets import load_dataset, load_metric
import numpy as np

model_path = "Factiverse/XLMR-large-XNLI-stance-downsampled"
onnx_path = Path('onnx/')
onnx_path.mkdir(exist_ok=True)

model = ORTModelForSequenceClassification.from_pretrained(model_path,export=True) #' export model to cuda if needed here'

quantizer = ORTQuantizer.from_pretrained(model)
qconfig = QuantizationConfig(
    is_static=False,
    format=QuantFormat.QOperator,  # Same as AutoQuantizationConfig.avx512_vnni for dynamic quantization
    mode=QuantizationMode.IntegerOps,  # Same as AutoQuantizationConfig.avx512_vnni for dynamic quantization
    per_channel=False,
    reduce_range=True,
    operators_to_quantize=["MatMul", "Add"],  # Same as AutoQuantizationConfig.avx512_vnni for dynamic quantization
)
quantizer.quantize(
    save_dir="XLMR-large-XNLI-stance-downsampled",
    quantization_config=qconfig,
)

onnx_model = ORTModelForSequenceClassification.from_pretrained("XLMR-large-XNLI-stance-downsampled",file_name="model_quantized.onnx")

eval_dataset = load_dataset("xnli", "en", split="validation")
task_evaluator = evaluator("text-classification")

def preprocess_function(example):
    example["input"] =[
            example["premise"],
            example["hypothesis"]]
    return example



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/820 [00:00<?, ?B/s]

Framework not specified. Using pt to export the model.


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: XLMRobertaForSequenceClassification *****
Using framework PyTorch: 2.4.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> False
Saving external data to one file...
Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: False)
Quantizing model...
Saving quantized model at: XLMR-large-XNLI-stance-downsampled (external data format: False)
Configuration saved in XLMR-large-XNLI-stance-downsampled/ort_config.json


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

In [ ]:
def quantize_model(model_path: str, output_path: str) -> None:
    """Quantizes a model and saves it to a given output path.

    Args:
        model_path: The path to the model to quantize.
        output_path: The path to save the quantized model.
    """

    model = ORTModelForSequenceClassification.from_pretrained(
        model_path, export=True
    )

    quantizer = ORTQuantizer.from_pretrained(model)
    qconfig = QuantizationConfig(
        is_static=False,
        format=QuantFormat.QOperator,
        mode=QuantizationMode.IntegerOps,
        per_channel=False,
        reduce_range=True,
        operators_to_quantize=[
            "MatMul",
            "Add",
        ],
    )
    quantizer.quantize(
        save_dir=output_path,
        quantization_config=qconfig,
    )
    quantized_model_size = os.path.getsize(output_path / "model_quantized.onnx"")/(1024*1024)
    print("quantized model size", quantized_model_size)

In [ ]:
onnx_model = ORTModelForSequenceClassification.from_pretrained("factiverse_stance_detection_multilingual_quantized",file_name="model_quantized.onnx")
#onnx_model.push_to_hub("factiverse_stance_detection_multilingual_quantized","Factiverse/factiverse_stance_detection_ort_quantized",use_auth_token=True)


In [ ]:
onnx_model.push_to_hub("XLMR-large-XNLI-stance-downsampled","Factiverse/XLMR-large-XNLI-stance-downsampled_quantized",use_auth_token=True)


/usr/local/lib/python3.10/dist-packages/optimum/modeling_base.py:166: FutureWarning: The `use_auth_token` argument is deprecated and will be removed soon. Please use the `token` argument instead.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model_quantized.onnx:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

In [ ]:
!pip install onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 8.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Factiverse/XLMR-large-XNLI-stance-downsampled")
tokenizer_kwargs ={"padding":True}


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("Factiverse/XLMR-large-XNLI-stance-downsampled")
model_class = pipeline('text-classification', model=model, tokenizer=tokenizer,device="cuda", **tokenizer_kwargs)

In [ ]:
import json
with open("stance_test.json") as f:
  data = json.load(f)

data[0]

FileNotFoundError: [Errno 2] No such file or directory: 'stance_test.json'

In [ ]:
statrt_time = time.time()
predictions = []
ground_truth = []
for index, row in enumerate(data):
  model_predictions = model_class(row["text_a"] + "|" + row["text_b"])
  if model_predictions[0]["label"] == "SUPPORTS":
    predictions.append(1)
  else:
    predictions.append(0)
  ground_truth.append(row["labels"])
print("total time taken by unquantized model",time.time()-statrt_time )

NameError: name 'data' is not defined

In [ ]:
count = 0
for index, prediction in enumerate(predictions):
  if prediction == ground_truth[index]:
    count+=1
print("Accuracy",count/(len(predictions)))

ZeroDivisionError: division by zero

In [ ]:
print("hi")

hi


In [ ]:
print("Accuracy",count/(len(predictions)))

Accuracy 0.7711864406779662


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Factiverse/XLMR-large-stance-factisearch-data-balanced_quantized")
onnx_model = ORTModelForSequenceClassification.from_pretrained("Factiverse/XLMR-large-stance-factisearch-data-balanced_quantized",file_name="model_quantized.onnx")
onnx_classifier = pipeline('text-classification', model=onnx_model, tokenizer=tokenizer,device="cuda", **tokenizer_kwargs)

config.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

model_quantized.onnx:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

use_io_binding was set to False, setting it to True because it can provide a huge speedup on GPUs. It is possible to disable this feature manually by setting the use_io_binding attribute back to False.


In [ ]:
onnx_classifier = pipeline('text-classification', model=onnx_model, tokenizer=tokenizer,device="cuda", **tokenizer_kwargs)

use_io_binding was set to False, setting it to True because it can provide a huge speedup on GPUs. It is possible to disable this feature manually by setting the use_io_binding attribute back to False.


In [ ]:
statrt_time = time.time()
predictions_quant = []
ground_truth_quant = []
for index, row in enumerate(data):
  model_predictions = onnx_classifier(row["text_a"] + "|" + row["text_b"])
  if model_predictions[0]["label"] == "SUPPORTS":
    predictions_quant.append(1)
  else:
    predictions_quant.append(0)
  ground_truth_quant.append(row["labels"])
print("total time taken by quantized model",time.time()-statrt_time )

total time taken by quantized model 136.30763864517212


In [ ]:
count = 0
for index, prediction in enumerate(predictions_quant):
  if prediction == ground_truth_quant[index]:
    count+=1
print("Accuracy",count/(len(predictions_quant)))

Accuracy 0.7711864406779662


In [ ]:
import json
with open("arabic_numeric_claims.json") as f:
  data = json.load(f)

In [ ]:
data[0]

{'country_of_origin': 'france',
 'gpt3_summary': None,
 'label': 'False',
 'published': '2021-11-08',
 'url': 'https://factcheck.afp.com/ar/http%253A%252F%252Fdoc.afp.com%252F9R99LU-1',
 'crawled_date': '2024-03-24T18:12:49',
 'factchecker': 'خدمة تقصّي صحّة الأخبار باللغة العربيّة',
 'rating_val': '1',
 'lang': 'ar',
 'fact_source': 'afp',
 'supported': True,
 'label_original': 'خطأ',
 'claim': 'صورة تُظهر رفع صور ثلاث شخصيات فكرية وسياسية مسلمة في حيّ للأعمال في ماليزيا',
 'doc': 'يظهر في الصورة ما يبدو أنه حيّ للأعمال في وسط تجاريّ من عاصمة ذات مبان حديثة وناطحات سحاب، وقد رُفعت على واحد من المباني صور رئيس الحكومة الماليزيّ السابق مهاتير محمّد البالغ من العمر 96 عاماً، والفقيه الجزائريّ عبد الحميد بن باديس المتوفى عام 1940، والمفّكر الجزائريّ مالك بن نبيّ المتوفى عام 1973. ويُعدّ عبد الحميد بن باديس ومالك بن نبيّ من أعلام الفكر الإسلاميّ في القرن العشرين، فيما يُنظر إلى مهاتير محمّد على أنه من أعلام النهضة الاقتصاديّة في ماليزيا، الدولة ذات الغالبية المسلمة في جنوب شرق آسيا. وجاء ف

In [ ]:
len(data)

1329